In [ ]:
# 匯入套件
import requests
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import json
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


In [ ]:
# 設定要讀取的檔案路徑並讀入黨案

# today = date.today()
file_name = date.today() - timedelta(days=1)

full_path = f'C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\FlightData/{file_name}_FlightList.csv'

df_list = pd.read_csv(full_path)

In [3]:
## 函式庫

def find_tag(div_list, target_str:str):
    """用於尋找特定字串標籤的index"""
    target=0
    for i in div_list:
        if i.get_text() == target_str:
            break
        else:
            target += 1
    return target


def safe_extract(func):
    """判斷一個soup物件是否存在/有值，若沒有則回傳None"""
    try:
        return func()
    except (IndexError, AttributeError):
        return None


def gate_exist(soup):
    """判斷一個航班頁面中是否有到/離閘口資料"""
    gate = 0
    x = soup
    for i in x('div'):
        if '閘口' in i.text:
            gate = 1
            break
    return gate


def crawl_with_gate(soup, url):
    """當該班機有到/離閘門資料時使用的爬蟲"""
    flight_data = []

    # 班機基本資料。較容易在各網頁中出現差異，故先使用函式取得定位，再去取得資訊
    div_list = soup('div', class_='flightPageDataLabel')

    # 航班編號、機型、航空公司、飛行距離
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageIdent')[0].h1.text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '機型')]('div', class_='flightPageData')[0].text.strip().replace('\xa0', ' ')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '航空公司')]('div', class_='flightPageData')[0].text.strip().split('\n')[0]))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '距離')].span.text.strip().replace(',', '').split(' ')[1]))

    # 起飛機場、起飛城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[0]('span', class_='flightPageSummaryCity')[0].text.strip()))

    # 降落機場、降落城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[0]('span', class_='destinationCity')[0].text.strip()))

    # 起飛日期
    flight_data.append(safe_extract(lambda: soup('span', class_='flightPageSummaryDepartureDay')[0].text))

    # 預計/實際離開閘門時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[0].span.text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[0]('div', class_="flightPageDataActualTimeText")[0].text.strip().replace('\xa0', ' ').replace('\\n', '').replace('\\t', '')))
    
    # 預計/實際起飛時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[1]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[1]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    
    # 抵達日期
    flight_data.append(safe_extract(lambda: soup('span', class_='flightPageSummaryArrivalDay')[0].text))

    # 預計/實際降落時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[2]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[2]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

    # 預計/實際抵達閘門時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[3]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[3]('div', class_='flightPageDataActualTimeText')[0].span.text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

    # 紀錄該航班網址，若有需要可再重新訪問
    flight_data.append(url)

    return flight_data



def crawl_without_gate(soup, url):
    """當該班機沒有到/離閘門資料時使用的爬蟲"""
    flight_data = []

    # 班機基本資料。較容易在各網頁中出現差異，故先使用函式取得定位，再去取得資訊
    div_list = soup('div', class_='flightPageDataLabel')

    # 航班編號、機型、航空公司、飛行距離
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageIdent')[0].h1.text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '機型')]('div', class_='flightPageData')[0].text.strip().replace('\xa0', ' ')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '航空公司')]('div', class_='flightPageData')[0].text.strip().split('\n')[0]))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[find_tag(div_list, '距離')].span.text.strip().replace(',', '').split(' ')[1]))

    # 起飛機場、起飛城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[0]('span', class_='flightPageSummaryCity')[0].text.strip()))

    # 降落機場、降落城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[0]('span', class_='destinationCity')[0].text.strip()))

    # 起飛日期
    flight_data.append(safe_extract(lambda: soup('span', class_='flightPageSummaryDepartureDay')[0].text))

    # 預計/實際離開閘門時間
    flight_data.append(None)
    flight_data.append(None)
    
    # 預計/實際起飛時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[0]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[0]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    
    # 抵達日期
    flight_data.append(safe_extract(lambda: soup('span', class_='flightPageSummaryArrivalDay')[0].text))

    # 預計/實際降落時間
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[1]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[1]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

    # 預計/實際抵達閘門時間
    flight_data.append(None)
    flight_data.append(None)

    # 紀錄該航班網址，若有需要可再重新訪問
    flight_data.append(url)

    return flight_data


In [4]:
# 建立dataframe需要的data list
data = []

# 建立selenium連線
driver_path = './chromedriver.exe'
service = Service(driver_path)
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=service, options=chrome_options)
print('建立連線')

# 根據df_list中的link欄位跑回圈，逐一進入網頁取得html編碼
for url in df_list['link']:
    try:
        driver.get(url)

        # 網頁內有JavaScript動態生成內容，故設定等待網頁讀取完畢後再動作
        wait = WebDriverWait(driver, 15)
        element = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "flightPageSummaryDepartureDay"))
        )

        # 如果有cookie選項的話選取同意，若沒有就跳過
        try:
            driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()
            time.sleep(4)

        except:
            pass

        # 取得網頁html碼，並轉換成soup物件
        page_source = driver.page_source
    
    except Exception as e:
        print(f"無法存取 {url}: {e}")
        continue

    soup = BeautifulSoup(page_source, 'html.parser')
    flight_no = soup('div', class_='flightPageIdent')[0].h1.text.strip()

    ## 根據取得的soup物件，開始抓取各項資訊
    print(f'開始查詢{flight_no}班機資訊資訊...')

    try:
        if gate_exist(soup) == 1:
            print(f'{flight_no}航班有到/離閘門資料')
            flight_data = crawl_with_gate(soup, url)
        
        else:
            print(f'{flight_no}航班沒有到/離閘門資料')
            flight_data = crawl_without_gate(soup, url)
    
    except Exception as e:
        print(f'發生錯誤：{e}')

    data.append(flight_data)
    print(f'完成存取{flight_no}航班資料')
    time.sleep(7)

driver.quit()

建立連線
開始查詢EVA11班機資訊資訊...
EVA11航班有到/離閘門資料
完成存取EVA11航班資料
開始查詢EVA119班機資訊資訊...
EVA119航班有到/離閘門資料
完成存取EVA119航班資料
開始查詢EVA12班機資訊資訊...
EVA12航班有到/離閘門資料
完成存取EVA12航班資料
開始查詢EVA15班機資訊資訊...
EVA15航班有到/離閘門資料
完成存取EVA15航班資料
開始查詢EVA159班機資訊資訊...
EVA159航班有到/離閘門資料
完成存取EVA159航班資料
開始查詢EVA17班機資訊資訊...
EVA17航班有到/離閘門資料
完成存取EVA17航班資料
開始查詢EVA171班機資訊資訊...
EVA171航班有到/離閘門資料
完成存取EVA171航班資料
開始查詢EVA18班機資訊資訊...
EVA18航班有到/離閘門資料
完成存取EVA18航班資料
開始查詢EVA188班機資訊資訊...
EVA188航班有到/離閘門資料
完成存取EVA188航班資料
開始查詢EVA195班機資訊資訊...
EVA195航班有到/離閘門資料
完成存取EVA195航班資料
開始查詢EVA205班機資訊資訊...
EVA205航班有到/離閘門資料
完成存取EVA205航班資料
開始查詢EVA238班機資訊資訊...
EVA238航班有到/離閘門資料
完成存取EVA238航班資料
開始查詢EVA25班機資訊資訊...
EVA25航班有到/離閘門資料
完成存取EVA25航班資料
開始查詢EVA256班機資訊資訊...
EVA256航班有到/離閘門資料
完成存取EVA256航班資料
開始查詢EVA261班機資訊資訊...
EVA261航班有到/離閘門資料
完成存取EVA261航班資料
開始查詢EVA27班機資訊資訊...
EVA27航班有到/離閘門資料
完成存取EVA27航班資料
開始查詢EVA31班機資訊資訊...
EVA31航班有到/離閘門資料
完成存取EVA31航班資料
開始查詢EVA316班機資訊資訊...
EVA316航班有到/離閘門資料
完成存取EVA316航班資料
開始查詢EVA32班機資訊資訊...
EVA32航班有到/離閘門資料
完成存取EVA32航班資料
開始查詢EVA35班機資訊資訊...
EVA35航班有到/離閘門資料

In [5]:
columns = [
    'flight_NO',
    'flight_type',
    'flight_company',
    'fly_distance',
    'departure_airport_code',
    'departure_city',
    'arrival_airport_code',
    'arrival_city',
    'departure_date',
    'leave_gate_estimate',
    'leave_gate_actual',
    'departure_time_estimate',
    'departure_time_actual',
    'arrival_date',
    'landing_time_estimate',
    'landing_time_actual',
    'arrive_gate_estimate',
    'arrive_gate_actual',
    'link'
]

df = pd.DataFrame(columns=columns, data=data)

In [10]:
df

,flight_NO,flight_type,flight_company,fly_distance,departure_airport_code,departure_city,arrival_airport_code,arrival_city,departure_date,leave_gate_estimate,leave_gate_actual,departure_time_estimate,departure_time_actual,arrival_date,landing_time_estimate,landing_time_actual,arrive_gate_estimate,arrive_gate_actual,link
0,EVA11,BOEING 777-300ER (雙發) (B77W),EVA Air,11236,LAX,"Los Angeles, CA",TPE,台灣臺北 TW,2025年 10月 06日 (星期一),00:15 PDT,00:19 PDT,00:25 PDT,00:42 PDT,2025年 10月 07日 (星期二),04:18 CST (+1),04:36 CST (+1),05:10 CST (+1),04:47 CST (+1),https://www.flightaware.com/live/flight/id/EVA...
1,EVA119,Airbus A321 (雙發) (A321),EVA Air,1696,FUK,福岡市 JP,KHH,"Kaohsiung City, Taiwan",2025年 10月 06日 (星期一),20:20 JST,20:19 JST,20:30 JST,20:32 JST,2025年 10月 06日 (星期一),21:38 CST,21:44 CST,22:10 CST,21:53 CST,https://www.flightaware.com/live/flight/id/EVA...
2,EVA12,BOEING 777-300ER (雙發) (B77W),EVA Air,11190,TPE,台灣臺北 TW,LAX,"Los Angeles, CA",2025年 10月 06日 (星期一),19:40 CST,19:39 CST,19:50 CST,19:55 CST,2025年 10月 06日 (星期一),16:07 PDT,16:34 PDT,16:40 PDT,16:41 PDT,https://www.flightaware.com/live/flight/id/EVA...
3,EVA15,BOEING 777-300ER (雙發) (B77W),EVA Air,11273,LAX,"Los Angeles, CA",TPE,台灣臺北 TW,2025年 10月 06日 (星期一),00:50 PDT,00:48 PDT,01:00 PDT,01:08 PDT,2025年 10月 07日 (星期二),05:08 CST (+1),05:13 CST (+1),05:45 CST (+1),05:21 CST (+1),https://www.flightaware.com/live/flight/id/EVA...
4,EVA159,BOEING 787-10 Dreamliner (雙發) (B78X),EVA Air,1609,ICN,韩国 首尔仁川 KR,TPE,台灣臺北 TW,2025年 10月 06日 (星期一),19:45 KST,20:32 KST,19:55 KST,21:09 KST,2025年 10月 06日 (星期一),20:55 CST,22:14 CST,21:25 CST,22:21 CST,https://www.flightaware.com/live/flight/id/EVA...
5,EVA17,BOEING 777-300ER (雙發) (B77W),EVA Air,10751,SFO,"San Francisco, CA",TPE,台灣臺北 TW,2025年 10月 06日 (星期一),01:10 PDT,01:25 PDT,01:20 PDT,01:43 PDT,2025年 10月 07日 (星期二),04:26 CST (+1),05:07 CST (+1),05:40 CST (+1),05:07 CST (+1),https://www.flightaware.com/live/flight/id/EVA...
6,EVA171,Airbus A321 (雙發) (A321),EVA Air,1843,ICN,韩国 首尔仁川 KR,KHH,"Kaohsiung City, Taiwan",2025年 10月 06日 (星期一),20:45 KST,21:11 KST,20:55 KST,21:41 KST,2025年 10月 06日 (星期一),22:13 CST,23:04 CST,22:40 CST,23:13 CST,https://www.flightaware.com/live/flight/id/EVA...
7,EVA18,BOEING 777-300ER (雙發) (B77W),EVA Air,10669,TPE,台灣臺北 TW,SFO,"San Francisco, CA",2025年 10月 06日 (星期一),19:40 CST,19:36 CST,19:50 CST,19:56 CST,2025年 10月 06日 (星期一),15:38 PDT,15:58 PDT,16:00 PDT,16:08 PDT,https://www.flightaware.com/live/flight/id/EVA...
8,EVA188,Boeing 737-800 (雙發) (B738),EVA Air,1009,SZX,深圳市 CN,TPE,台灣臺北 TW,2025年 10月 06日 (星期一),19:35 CST,20:17 CST,19:45 CST,20:47 CST,2025年 10月 06日 (星期一),21:10 CST,22:17 CST,21:35 CST,22:17 CST,https://www.flightaware.com/live/flight/id/EVA...
9,EVA195,BOEING 787-10 Dreamliner (雙發) (B78X),EVA Air,2335,NRT,日本 东京成田 JP,TPE,台灣臺北 TW,2025年 10月 06日 (星期一),20:40 JST,20:47 JST,20:50 JST,21:04 JST,2025年 10月 06日 (星期一),22:42 CST,22:58 CST,23:20 CST,23:02 CST,https://www.flightaware.com/live/flight/id/EVA...
